# PySpark and Hadoop

In this notebook we look at real data while using our small hadoop cluster. For programming we will use parallel Spark dataframes with real data.

We look at [the New York City Taxi Cab dataset](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). This includes every ride made in the city of New York in the year 2016.

On [this website](http://chriswhong.github.io/nyctaxi/) you can see the data for one random NYC yellow taxi on a single day.

On [this post](http://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/), you can see an analysis of this dataset. Postgres and R scripts are available on [GitHub](https://github.com/toddwschneider/nyc-taxi-data).

## Loading the data

Normally we would read and load this data into memory as a Pandas dataframe.  However in this case that would be unwise because this data is too large to fit in RAM.

The data can stay in the hdfs filesystem but for performance reason we can't use the csv format. The file is large (32Go) and text formatted. Data Access is very slow.

## Parquet format

[Parquet format](https://github.com/apache/parquet-format) is a common binary data store, used particularly in the Hadoop/big-data sphere. It provides several advantages relevant to big-data processing:

- columnar storage, only read the data of interest
- efficient binary packing
- choice of compression algorithms and encoding
- split data into files, allowing for parallel processing
- range of logical types
- statistics stored in metadata allow for skipping unneeded chunks
- data partitioning using the directory structure

To convert the csv file to parquet we can use Dask or Spark. Here the code using Spark.
```python
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master('local[16]') \
        .config('spark.hadoop.parquet.enable.summary-metadata', 'true') \
        .getOrCreate()
df = spark.read.csv(
    "hdfs://localhost:54310/user/pnavaro/2016_Yellow_Taxi_Trip_Data.csv", 
                    header="true",inferSchema="true")
df.write.parquet('hdfs://localhost:54310/user/pnavaro/nyc-taxi/2016.parquet')
spark.stop()
```

## hdfs3

[hdfs3](http://hdfs3.readthedocs.io/en/latest/) is an alternative native C/C++ HDFS client that interacts with HDFS without the JVM, exposing first class support to non-JVM languages like Python.

This library, hdfs3, is a lightweight Python wrapper around the C/C++ libhdfs3 library. It provides both direct access to libhdfs3 from Python as well as a typical Pythonic interface.

In [7]:
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(host='localhost', port=54310)

In [8]:
from lorem import paragraph
with hdfs.open('/user/pnavaro/samples.txt','wb') as f:
    f.write(paragraph())

In [9]:
hdfs.ls('/user/pnavaro')

['/user/pnavaro/1990.csv',
 '/user/pnavaro/1991.csv',
 '/user/pnavaro/1992.csv',
 '/user/pnavaro/1993.csv',
 '/user/pnavaro/1994.csv',
 '/user/pnavaro/1995.csv',
 '/user/pnavaro/1996.csv',
 '/user/pnavaro/1997.csv',
 '/user/pnavaro/1998.csv',
 '/user/pnavaro/1999.csv',
 '/user/pnavaro/2016_Yellow_Taxi_Trip_Data.csv',
 '/user/pnavaro/copied-file.txt',
 '/user/pnavaro/nyc-taxi',
 '/user/pnavaro/nycflights.parquet',
 '/user/pnavaro/remote-file.txt',
 '/user/pnavaro/samples.txt']

In [10]:
from lorem import text
with open('local-file.txt','w') as f:
    f.write(text())
hdfs.put('local-file.txt', '/user/pnavaro/remote-file.txt')

In [11]:
hdfs.ls('/user/pnavaro/')

['/user/pnavaro/1990.csv',
 '/user/pnavaro/1991.csv',
 '/user/pnavaro/1992.csv',
 '/user/pnavaro/1993.csv',
 '/user/pnavaro/1994.csv',
 '/user/pnavaro/1995.csv',
 '/user/pnavaro/1996.csv',
 '/user/pnavaro/1997.csv',
 '/user/pnavaro/1998.csv',
 '/user/pnavaro/1999.csv',
 '/user/pnavaro/2016_Yellow_Taxi_Trip_Data.csv',
 '/user/pnavaro/copied-file.txt',
 '/user/pnavaro/nyc-taxi',
 '/user/pnavaro/nycflights.parquet',
 '/user/pnavaro/remote-file.txt',
 '/user/pnavaro/samples.txt']

In [12]:
hdfs.mv('/user/pnavaro/remote-file.txt', '/user/pnavaro/copied-file.txt')

False

In [1]:
hdfs.ls('/user/pnavaro')

NameError: name 'hdfs' is not defined

## Exercise

We want to implement the WordCount application already coded in notebook 01.MapReduce. 
This time, files are 1000 times bigger and stored in hadoop file system.

### Prepare data and put it on hdfs

In [51]:
from lorem import text
with open('sample.txt','w') as f:
    for i in range(2250):
        f.write(text())

In [52]:
%%file cp_file.sh
# Makes n number of copies of sample.txt
mkdir -p data/latin
INPUT=sample.txt
for num in $(seq 1 1000)
do
    bn=$(basename $INPUT .txt)
    cp $INPUT data/latin/$bn$num.txt
done

Overwriting cp_file.sh


In [53]:
!chmod +x cp_file.sh; ./cp_file.sh

In [54]:
!du -sh data/latin/

3,0G	data/latin/


- Put the sample txt files on hadoop

### Get filenames and files size
- Change the functions below by using hdfs3 to get filenames and size on HDFS.
- Take a look at the [API](http://hdfs3.readthedocs.io/en/latest/api.html)

In [64]:
import os
def get_filenames(root):
	"""
	Returns complete list of filenames in root directory
    """

	files = []
	for f in os.listdir(root):
		if f.endswith(".txt"):
			files.append(f)
	
	return files

root = os.path.join(os.getcwd(),'data','latin')
get_filenames(root)

['sample1.txt',
 'sample10.txt',
 'sample100.txt',
 'sample1000.txt',
 'sample101.txt',
 'sample102.txt',
 'sample103.txt',
 'sample104.txt',
 'sample105.txt',
 'sample106.txt',
 'sample107.txt',
 'sample108.txt',
 'sample109.txt',
 'sample11.txt',
 'sample110.txt',
 'sample111.txt',
 'sample112.txt',
 'sample113.txt',
 'sample114.txt',
 'sample115.txt',
 'sample116.txt',
 'sample117.txt',
 'sample118.txt',
 'sample119.txt',
 'sample12.txt',
 'sample120.txt',
 'sample121.txt',
 'sample122.txt',
 'sample123.txt',
 'sample124.txt',
 'sample125.txt',
 'sample126.txt',
 'sample127.txt',
 'sample128.txt',
 'sample129.txt',
 'sample13.txt',
 'sample130.txt',
 'sample131.txt',
 'sample132.txt',
 'sample133.txt',
 'sample134.txt',
 'sample135.txt',
 'sample136.txt',
 'sample137.txt',
 'sample138.txt',
 'sample139.txt',
 'sample14.txt',
 'sample140.txt',
 'sample141.txt',
 'sample142.txt',
 'sample143.txt',
 'sample144.txt',
 'sample145.txt',
 'sample146.txt',
 'sample147.txt',
 'sample148.txt'

In [75]:
def files_total_size(root):
   """
   Prints sum of filesize of files to be mapreduced
   """
   filesize = 0
   for f in os.listdir(root):
       if f.endswith(".txt"):
           filesize += os.path.getsize(os.path.join(root,f))

   print("Size of files:", filesize / 1073741824, "GB")

files_total_size(root)

Size of files: 2.9646381735801697 GB


[fastparquet](http://fastparquet.readthedocs.io/en/latest/) provides a performant library to read and write Parquet files from Python, without any need for a Python-Java bridge. This will make the Parquet format an ideal storage mechanism for Python-based big data workflows.

The tabular nature of Parquet is a good fit for the Pandas data-frame objects, and we exclusively deal with data-frame<->Parquet.

[Apache Arrow](https://arrow.apache.org/docs/python/)

Arrow is a columnar in-memory analytics layer designed to accelerate big data. It houses a set of canonical in-memory representations of flat and hierarchical data along with multiple language-bindings for structure manipulation.

https://arrow.apache.org/docs/python/parquet.html

The Apache Parquet project provides a standardized open-source columnar storage format for use in data analysis systems. It was created originally for use in Apache Hadoop with systems like Apache Drill, Apache Hive, Apache Impala (incubating), and Apache Spark adopting it as a shared standard for high performance data IO.

Apache Arrow is an ideal in-memory transport layer for data that is being read or written with Parquet files. [PyArrow](https://arrow.apache.org/docs/python/) includes Python bindings to read and write Parquet files with pandas.



In [15]:
import fastparquet as fp
import pyarrow as pa
import pyarrow.parquet as pq

hdfs = pa.hdfs.connect('localhost', 54310, 'pnavaro', driver='libhdfs3')

pf = fp.ParquetFile('/user/pnavaro/nyc-taxi/2016.parquet', open_with=hdfs.open)
pf

<Parquet File: {'name': '/user/pnavaro/nyc-taxi/2016.parquet/_metadata', 'columns': ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'PULocationID', 'DOLocationID'], 'partitions': [], 'rows': 131165043}>

In [ ]:
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.master('spark://schedulers:7077').getOrCreate()
#spark

In [16]:
columns = ['tpep_pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'tip_amount', 'total_amount']

df = (spark.read.parquet('hdfs://localhost:54310/user/pnavaro/nyc-taxi/2016.parquet')
           .select(*columns))

- Sum the total number of passengers

In [17]:
df.agg({'passenger_count': 'sum'}).collect()

[Row(sum(passenger_count)=217355302)]

In [18]:
df.agg({'passenger_count': 'avg'}).collect()

[Row(avg(passenger_count)=1.6571130312517794)]

In [19]:
df.groupby('passenger_count').agg({'*': 'count'}).collect()

[Row(passenger_count=1, count(1)=92987719),
 Row(passenger_count=6, count(1)=4234423),
 Row(passenger_count=3, count(1)=5456807),
 Row(passenger_count=5, count(1)=6773026),
 Row(passenger_count=9, count(1)=261),
 Row(passenger_count=4, count(1)=2660369),
 Row(passenger_count=8, count(1)=316),
 Row(passenger_count=7, count(1)=361),
 Row(passenger_count=2, count(1)=19038307),
 Row(passenger_count=0, count(1)=13454)]

## Exercise

Use the `filter`, `groupBy`, `agg` operations to find out how well New Yorkers tip based on the number of passengers in the cab.
 1. Remove rides with zero fare
 2. Add a new column tip_fraction that is equal to the ratio of the tip to the fare
 3. Group by the passenger_count column and take the mean of the tip_fraction column.
 
You may want to refer to these resources to help you with the Spark DataFrame API
- https://spark.apache.org/docs/latest/api/python/pyspark.sql.html
- https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf
And refer to the Spark UI for feedback.

### How to remove rows

In Spark you can filter rows by a boolean expression like the following:
```python
df.filter(df.name == 'Alice')
```

### How to make new columns

In Pandas you can create a new column using Python's setitem syntax like the following:

```python
df = df.withColumn('z', df.x + df.y)
```

### How to do groupby-aggregations

In Pandas you can do a groupby-aggregation by using the `groupby` method, followed by a column name an aggregation method like the following:

```python
df.groupBy(df.name).agg({'column-name': 'avg'})
```

When you want to collect the result of your computation, finish with the `.collect()` method.

### Exercises

1. How well do New Yorkers tip as a function of the hour of day and the day of the week?
2. Investiate the `payment_type` column.  See how well each of the payment types correlate with the `tip_fraction`.  Did you find anything interesting?  Any guesses on what the different payment types might be?  If you're interested you may be able to find more information on the [NYC TLC's website](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)
3. How quickly can you get the data for a particular day of the year?  How about for a particular hour of that day?



In [ ]:
spark.stop()